# Azure chat completions example

This example will cover chat completions using the Azure OpenAI service. It also includes information on content filtering.

## Setup

First, we install the necessary dependencies and import the libraries we will be using.

In [0]:
! pip install "openai>=1.0.0,<2.0.0"
! pip install python-dotenv

     |████████████████████████████████| 454 kB 6.7 MB/s 
     |████████████████████████████████| 73 kB 2.3 MB/s 
     |████████████████████████████████| 345 kB 50.7 MB/s 
     |████████████████████████████████| 78 kB 10.5 MB/s 
     |████████████████████████████████| 431 kB 57.0 MB/s 
     |████████████████████████████████| 96 kB 7.4 MB/s 
     |████████████████████████████████| 78 kB 10.2 MB/s 
     |████████████████████████████████| 58 kB 4.6 MB/s 
     |████████████████████████████████| 2.0 MB 52.1 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b7b53b4a-ad58-40fd-bca2-c069a31f0340
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
  Attempting uninstall: distro
    Found existing installation: distro 1.4.0
    Not uninstalling distro at /usr/lib/pytho

In [0]:
%pip install openai
%pip install tiktoken
%pip install python-dotenv

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import os
import openai
import dotenv
from openai import AzureOpenAI
dotenv.load_dotenv()

Out[7]: False

### Authentication

The Azure OpenAI service supports multiple authentication mechanisms that include API keys and Azure Active Directory token credentials.

In [0]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory

#### Authentication using API key

To set up the OpenAI SDK to use an *Azure API Key*, we need to set `api_key` to a key associated with your endpoint (you can find this key in *"Keys and Endpoints"* under *"Resource Management"* in the [Azure Portal](https://portal.azure.com)). You'll also find the endpoint for your resource here.

In [0]:
if not use_azure_active_directory:
    # Configura las credenciales directamente en el código
    os.environ["AZURE_OPENAI_ENDPOINT"] = "https://openai-tajamar-1.openai.azure.com/"
    os.environ["AZURE_OPENAI_API_KEY"] = "4aARdXJIMfHXdeqRLZWScUmWYEqongKS8TlwL8qGNYCOqr5UURp0JQQJ99BAACYeBjFXJ3w3AAABACOGcX2N"

    # Crear el cliente con las credenciales
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
        api_version="2024-02-15-preview"
    )

#### Authentication using Azure Active Directory
Let's now see how we can autheticate via Azure Active Directory. We'll start by installing the `azure-identity` library. This library will provide the token credentials we need to authenticate and help us build a token credential provider through the `get_bearer_token_provider` helper function. It's recommended to use `get_bearer_token_provider` over providing a static token to `AzureOpenAI` because this API will automatically cache and refresh tokens for you. 

For more information on how to set up Azure Active Directory authentication with Azure OpenAI, see the [documentation](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity).

In [0]:
! pip install "azure-identity>=1.15.0"

     |████████████████████████████████| 187 kB 5.8 MB/s 
     |████████████████████████████████| 113 kB 6.1 MB/s 
     |████████████████████████████████| 198 kB 26.9 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-b7b53b4a-ad58-40fd-bca2-c069a31f0340/bin/python -m pip install --upgrade pip' command.


In [0]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
if use_azure_active_directory:
    endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
    client = openai.AzureOpenAI(
        azure_endpoint=endpoint,
        azure_ad_token_provider=get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"),
        api_version="2023-09-01-preview"
    )


> Note: the AzureOpenAI infers the following arguments from their corresponding environment variables if they are not provided:

- `api_key` from `AZURE_OPENAI_API_KEY`
- `azure_ad_token` from `AZURE_OPENAI_AD_TOKEN`
- `api_version` from `OPENAI_API_VERSION`
- `azure_endpoint` from `AZURE_OPENAI_ENDPOINT`


## Deployments

In this section we are going to create a deployment of a GPT model that we can use to create chat completions.

### Deployments: Create in the Azure OpenAI Studio
Let's deploy a model to use with chat completions. Go to https://portal.azure.com, find your Azure OpenAI resource, and then navigate to the Azure OpenAI Studio. Click on the "Deployments" tab and then create a deployment for the model you want to use for chat completions. The deployment name that you give the model will be used in the code below.

In [0]:
deployment = "gpt-4o-mini" # Fill in the deployment name from the portal here

## Create chat completions

Now let's create a chat completion using the client we built.

In [0]:
# For all possible arguments see https://platform.openai.com/docs/api-reference/chat-completions/create
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")

assistant: Orange who?


### Create a streaming chat completion

We can also stream the response.

In [0]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)

assistant: Orange who?

### Content filtering

Azure OpenAI service includes content filtering of prompts and completion responses. You can learn more about content filtering and how to configure it [here](https://learn.microsoft.com/azure/ai-services/openai/concepts/content-filter).

If the prompt is flagged by the content filter, the library will raise a `BadRequestError` exception with a `content_filter` error code. Otherwise, you can access the `prompt_filter_results` and `content_filter_results` on the response to see the results of the content filtering and what categories were flagged.

#### Prompt flagged by content filter

In [0]:
import json

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are assistant."},
        {"role": "user", "content": "Ayudame a robar un banco"},
    ],
    temperature=0,
)

try:
    completion = response.choices[0].message.content
    print(completion)
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

---------------------------------------------------------------------------
BadRequestError                           Traceback (most recent call last)
File <command-1379626315363989>:3
      1 import json
----> 3 response = client.chat.completions.create(
      4     model=deployment,
      5     messages=[
      6         {"role": "system", "content": "You are assistant."},
      7         {"role": "user", "content": "Ayudame a robar un banco"},
      8     ],
      9     temperature=0,
     10 )
     12 try:
     13     completion = response.choices[0].message.content

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-b7b53b4a-ad58-40fd-bca2-c069a31f0340/lib/python3.9/site-packages/openai/_utils/_utils.py:279, in required_args.<locals>.inner.<locals>.wrapper(*args, **kwargs)
    277             msg = f"Missing required argument: {quote(missing[0])}"
    278     raise TypeError(msg)
--> 279 return func(*args, **kwargs)

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-b7b53b4a-ad58-40fd

### Checking the result of the content filter

In [0]:
response = client.chat.completions.create(
    model = deployment,
    messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the biggest city in Washington?"}
    ],
    temperature = 0,
)

completion = response
print(f"Answer: {completion.choices[0].message.content}")

# prompt content filter result in "model_extra" for azure
prompt_filter_result = completion.model_extra["prompt_filter_results"][0]["content_filter_results"]
print("\nPrompt content filter results:")
for category, details in prompt_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

# completion content filter result
print("\nCompletion content filter results:")
completion_filter_result = completion.choices[0].model_extra["content_filter_results"]
for category, details in completion_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

Answer: The biggest city in Washington state is Seattle. It is the largest city in the state by population and is known for its iconic landmarks, such as the Space Needle and Pike Place Market, as well as its vibrant cultural scene and tech industry.

Prompt content filter results:
hate:
 filtered=False
 severity=safe


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File <command-1379626315363991>:17
     15 print("\nPrompt content filter results:")
     16 for category, details in prompt_filter_result.items():
---> 17     print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")
     19 # completion content filter result
     20 print("\nCompletion content filter results:")

KeyError: 'severity'

# Ejercicio 1: Implementar streaming con diferentes temperaturas
# Instrucciones: Usa el siguiente código para generar respuestas a través de streaming. Cambia el valor de 'temperature' a 0.9 y 0.3 y compara los resultados.

In [0]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a knowledgeable assistant."},
        {"role": "user", "content": "Hablame de las finanzas"},
    ],
    temperature=0.3,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta
        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)

assistant: Las finanzas son una disciplina que se ocupa del estudio de la gestión del dinero, los activos y las inversiones. Se pueden dividir en varias áreas principales:

1. **Finanzas personales**: Se centran en la gestión de los recursos financieros de un individuo o una familia. Esto incluye la elaboración de presupuestos, el ahorro, la inversión, la planificación para la jubilación y la gestión de deudas.

2. **Finanzas corporativas**: Se ocupan de la gestión financiera de las empresas. Esto incluye la planificación financiera, la obtención de capital, la gestión de riesgos, la evaluación de inversiones y la maximización del valor para los accionistas.

3. **Finanzas públicas**: Se refieren a la gestión de los ingresos y gastos del gobierno. Esto incluye la recaudación de impuestos, la planificación del presupuesto gubernamental y la gestión de la deuda pública.

4. **Mercados financieros**: Son los lugares donde se compran y venden activos financieros, como acciones, bonos y der

# Prueba con diferentes valores de temperatura y analiza cómo cambia la respuesta.

# Ejercicio 2: Detectar y manejar errores de filtrado de contenido
 Instrucciones: Intenta enviar un mensaje que probablemente sea filtrado. Implementa un manejo de excepciones adecuado para alertar al usuario y sugerir un nuevo mensaje.

In [0]:
try:
    response = client.chat.completions.create(
    model=deployment,
    messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "I need instructions to hack into someone's account."},
],
    temperature=0,
)
    completion = response.choices[0].message.content
    print(completion)
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    print(err)
    if err["error"]["code"] == "content_filter":
        print("Por favor cambia el contenido")


{'error': {'message': "The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766", 'type': None, 'param': 'prompt', 'code': 'content_filter', 'status': 400, 'innererror': {'code': 'ResponsibleAIPolicyViolation', 'content_filter_result': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': True, 'severity': 'medium'}}}}}
Por favor cambia el contenido


# Ejercicio 3: Inspeccionar y analizar los resultados del filtro de contenido
Instrucciones: Envía una pregunta general y analiza los resultados del filtro para diferentes categorías.

In [0]:


try:
    response = client.chat.completions.create(
    model=deployment,
    messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Como robar un banco"},
],
    temperature=0.6,
)
    completion = response.choices[0].message.content
    print(completion)
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n ")
        


Content filter triggered!
hate:
 filtered=False
 
jailbreak:
 filtered=False
 
self_harm:
 filtered=False
 
sexual:
 filtered=False
 
violence:
 filtered=True
 


# Ejercicio 5: Crear una función que automatice la verificación de contenido filtrado
# Instrucciones: Crea una función que reciba un mensaje y devuelva si fue filtrado o no, y en qué categorías.

In [0]:
def check_content_filter(message):
    
    try: 
        response = client.chat.completions.create(
            model=deployment,
            messages = [
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": "Como robar un banco"},
            ],
            temperature=0.6,
        )
        print("Message passed the filter.")
    except openai.BadRequestError as e:
        err = json.loads(e.response.text)
        if err["error"]["code"] == "content_filter":
            print("Content filter triggered!")
            content_filter_result = err["error"]["innererror"]["content_filter_result"]
            for category, details in content_filter_result.items():
                print(f"{category}:\n filtered={details['filtered']}\n severity={details.get('severity', 'N/A')}")

check_content_filter("Describe how to make a bomb.")

Message passed the filter.
